In [5]:
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# API 키 가져오기
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

# 환경 변수 설정 (필요한 경우만)
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

# 확인 (보안상 일부만 출력)
print(f"OPENAI_API_KEY: {OPENAI_API_KEY[:4]}****")
print(f"TAVILY_API_KEY: {TAVILY_API_KEY[:4]}****")

OPENAI_API_KEY: sk-p****
TAVILY_API_KEY: tvly****


In [6]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain.tools import tool
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType

# ✅ Step 1: cafe_menu.txt 생성
menu_text = """1. 아메리카노
• 가격: ₩4,500
• 주요 원료: 에스프레소, 뜨거운 물
• 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.

2. 카페라떼
• 가격: ₩5,500
• 주요 원료: 에스프레소, 스팀 밀크
• 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다. 크리미한 질감과 부드러운 맛이 특징이며, 다양한 시럽과 토핑 추가가 가능합니다. 라떼 아트로 시각적 즐거움도 제공합니다.

...(중략)...

10. 티라미수
• 가격: ₩7,500
• 주요 원료: 마스카포네 치즈, 에스프레소, 레이디핑거, 코코아 파우더
• 설명: 이탈리아 전통 디저트로 마스카포네 치즈와 에스프레소에 적신 레이디핑거를 층층이 쌓아 만들었습니다. 부드럽고 달콤한 맛이 특징이며, 코코아 파우더로 마무리하여 깊은 풍미를 자랑합니다.
"""
with open("./data/cafe_menu.txt", "w", encoding="utf-8") as f:
    f.write(menu_text)

# ✅ Step 2: 벡터 DB 생성
loader = TextLoader("./data/cafe_menu.txt", encoding="utf-8")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
split_docs = splitter.split_documents(docs)

embeddings = OllamaEmbeddings(model="bge-m3")
db = FAISS.from_documents(split_docs, embeddings)
db.save_local("./db/cafe_db")

# ✅ Step 3: 도구 정의
tavily_tool = TavilySearchResults()
wiki_wrapper = WikipediaAPIWrapper()

@tool
def wiki_summary(query: str) -> str:
    """위키피디아에서 주제에 대한 요약 정보를 검색합니다."""
    return wiki_wrapper.run(query)

@tool
def db_search_cafe_func(query: str):
    """로컬 카페 메뉴 DB에서 관련 메뉴 정보를 검색합니다."""
    db = FAISS.load_local(
        "./db/cafe_db",
        embeddings,
        allow_dangerous_deserialization=True  # ✅ 중요: FAISS 로드 보안 예외 허용
    )
    results = db.similarity_search(query, k=2)
    return "\n".join([doc.page_content for doc in results])

# ✅ Step 4: LLM Agent 설정 및 실행
llm = ChatOpenAI(temperature=0.2)

agent_executor = initialize_agent(
    tools=[tavily_tool, wiki_summary, db_search_cafe_func],
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

# ✅ Step 5: 실행 테스트
question = "아메리카노의 가격과 특징은 무엇인가요?"
response = agent_executor.invoke({"input": question})

print("\n[✅ 최종 응답]")
print(response["output"])




> Entering new AgentExecutor chain...

Invoking: `db_search_cafe_func` with `{'query': '아메리카노'}`


1. 아메리카노
• 가격: ₩4,500
• 주요 원료: 에스프레소, 뜨거운 물
• 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.
...(중략)...

10. 티라미수
• 가격: ₩7,500
• 주요 원료: 마스카포네 치즈, 에스프레소, 레이디핑거, 코코아 파우더
• 설명: 이탈리아 전통 디저트로 마스카포네 치즈와 에스프레소에 적신 레이디핑거를 층층이 쌓아 만들었습니다. 부드럽고 달콤한 맛이 특징이며, 코코아 파우더로 마무리하여 깊은 풍미를 자랑합니다.아메리카노의 가격은 ₩4,500이며, 주요 원료는 에스프레소와 뜨거운 물입니다. 아메리카노는 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피로, 원두 본연의 맛을 가장 잘 느낄 수 있고 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽을 추가할 수도 있습니다.

> Finished chain.

[✅ 최종 응답]
아메리카노의 가격은 ₩4,500이며, 주요 원료는 에스프레소와 뜨거운 물입니다. 아메리카노는 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피로, 원두 본연의 맛을 가장 잘 느낄 수 있고 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽을 추가할 수도 있습니다.


In [8]:
# ✅ 필요한 모듈 임포트
from langchain_community.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
from langchain.tools import tool
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

# ✅ 벡터 DB 로드 (보안 옵션 포함)
embeddings = OllamaEmbeddings(model="bge-m3")
vector_db = FAISS.load_local(
    "./db/cafe_db",
    embeddings,
    allow_dangerous_deserialization=True  # 보안 허용
)

# ✅ 도구 정의
tavily_tool = TavilySearchResults()
wiki_wrapper = WikipediaAPIWrapper()

@tool
def wiki_summary(query: str) -> str:
    """위키피디아에서 주제에 대한 요약 정보를 검색합니다."""
    return wiki_wrapper.run(query)

@tool
def db_search_cafe_func(query: str):
    """로컬 카페 메뉴 DB에서 관련 메뉴 정보를 검색합니다."""
    results = vector_db.similarity_search(query, k=2)
    return "\n".join([doc.page_content for doc in results])

# ✅ Few-shot 프롬프트 구성
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(
        content="""당신은 카페 메뉴 정보와 일반 지식을 안내하는 AI 어시스턴트입니다.

[도구 사용 규칙]
- db_search_cafe_func: 메뉴 가격, 재료, 설명 등 카페 메뉴 정보
- wiki_summary: 역사, 문화적 배경, 제조 방식 등 일반 지식
- tavily_search_func: 트렌드, 뉴스 등 실시간 정보

[도구 사용 지침]
1. 메뉴 질문 → 반드시 db_search_cafe_func 사용
2. 역사/문화 지식 → wiki_summary 사용
3. 트렌드/실시간 뉴스 → tavily_search_func 사용
4. 복합 질문은 관련 도구를 순차적으로 호출
5. 결과는 출처별로 명확히 구분하여 정리"""
    ),
    HumanMessage(content="아메리카노 정보와 커피의 역사를 알려줘"),
    AIMessage(content="메뉴 정보와 위키 지식 검색을 함께 진행하겠습니다."),
    AIMessage(content="[db_search_cafe_func 결과]\n아메리카노는 ₩4,500이며, 에스프레소와 뜨거운 물로 구성된 클래식한 블랙 커피입니다."),
    AIMessage(content="[wiki_summary 결과]\n커피는 고대 에티오피아에서 유래하였고, 이후 중동과 유럽으로 전파되었습니다."),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessage(content="{input}")
])

# ✅ LLM + 체인 생성
llm = ChatOpenAI(temperature=0.2)
few_shot_chain = LLMChain(llm=llm, prompt=prompt)

# ✅ Agent 실행기 초기화
agent_executor = initialize_agent(
    tools=[tavily_tool, wiki_summary, db_search_cafe_func],
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

# ✅ 실행 함수 정의
def run_few_shot_cafe_assistant(user_question: str):
    print(f"\n[🧠 사용자 질문] {user_question}")
    result = agent_executor.invoke({"input": user_question})
    print("\n[✅ 최종 응답]")
    print(result["output"])

# ✅ 테스트 실행
run_few_shot_cafe_assistant("카페라떼와 어울리는 디저트는 뭐야? 그리고 라떼의 유래도 알려줘.")


C:\Users\user\AppData\Local\Temp\ipykernel_15996\886099571.py:63: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  few_shot_chain = LLMChain(llm=llm, prompt=prompt)



[🧠 사용자 질문] 카페라떼와 어울리는 디저트는 뭐야? 그리고 라떼의 유래도 알려줘.


> Entering new AgentExecutor chain...

Invoking: `db_search_cafe_func` with `{'query': '카페라떼와 어울리는 디저트'}`


2. 카페라떼
• 가격: ₩5,500
• 주요 원료: 에스프레소, 스팀 밀크
• 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다. 크리미한 질감과 부드러운 맛이 특징이며, 다양한 시럽과 토핑 추가가 가능합니다. 라떼 아트로 시각적 즐거움도 제공합니다.

...(중략)...
...(중략)...

10. 티라미수
• 가격: ₩7,500
• 주요 원료: 마스카포네 치즈, 에스프레소, 레이디핑거, 코코아 파우더
• 설명: 이탈리아 전통 디저트로 마스카포네 치즈와 에스프레소에 적신 레이디핑거를 층층이 쌓아 만들었습니다. 부드럽고 달콤한 맛이 특징이며, 코코아 파우더로 마무리하여 깊은 풍미를 자랑합니다.카페라떼와 어울리는 디저트로는 티라미수가 좋은 선택입니다. 티라미수는 이탈리아 전통 디저트로, 마스카포네 치즈와 에스프레소에 적신 레이디핑거를 층층이 쌓아 만들어지는 달콤하고 부드러운 맛을 가지고 있습니다. 라떼와 함께 즐기면 깊은 풍미를 느낄 수 있을 것입니다.

라떼의 유래에 대해 알려드리겠습니다. 라떼는 이탈리아어로 "우유"를 뜻하는 "latte"에서 유래하였습니다. 카페라떼는 에스프레소에 스팀밀크를 추가하여 만든 음료로, 부드럽고 크리미한 맛이 특징입니다. 라떼는 이탈리아를 비롯한 많은 나라에서 사랑받는 커피 음료 중 하나입니다.

> Finished chain.

[✅ 최종 응답]
카페라떼와 어울리는 디저트로는 티라미수가 좋은 선택입니다. 티라미수는 이탈리아 전통 디저트로, 마스카포네 치즈와 에스프레소에 적신 레이디핑거를 층층이 쌓아 만들어지는 달콤하고 부드러운 맛을 가지고 있습니다. 라떼와 함께 즐기면 깊은 풍미를 느낄 수 있을 것입니다.

라떼의